In [23]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.transform import resize 

## Reading Image

In [24]:
img=imread("images/img6.jpg")

In [25]:
img.shape

(254, 400, 3)

In [26]:
plt.figure()
plt.imshow(img[:,:,:3])
plt.show()

## Removing Perspective Distortion

In [27]:
def cross(v1,v2):
    A=np.array([[0,-v1[2],v1[1]],
                [v1[2],0,-v1[0]],
                [-v1[1],v1[0],0]])
    return A.dot(v2)

In [28]:
def remove_perspective(image):
    
    img=image[:,:,:3]
    %matplotlib qt
    plt.figure()
    plt.imshow(img)
    print("Select 4 points in Image for perspective distortion.")
    pts=plt.ginput(n=4,timeout=0)
    plt.show()
    
    p1=np.array([pts[0][0],pts[0][1],1])
    p2=np.array([pts[1][0],pts[1][1],1])
    p3=np.array([pts[2][0],pts[2][1],1])
    p4=np.array([pts[3][0],pts[3][1],1])
    
    
    l1=cross(p1,p2)
    l2=cross(p3,p4)
    l3=cross(p1,p4)
    l4=cross(p2,p3)
    l1=l1/l1[2];l2=l2/l2[2];l3=l3/l3[2];l4=l4/l4[2]
    
    
    v1=cross(l1,l2)
    v2=cross(l3,l4)
    v1=v1/v1[2];v2=v2/v2[2]
    l_v=cross(v1,v2)
    
    l_v=l_v/l_v[2]
    
    
    H=np.array([[1,0,0],
            [0,1,0],
            [l_v[0],l_v[1],l_v[2]]])
    
    

    
    return H,pts

In [ ]:
H,pts=remove_perspective(img)

Select 4 points in Image for perspective distortion.


## Drawing selected points

In [8]:
img1=img.copy()
lineThickness = 2
cv2.line(img1, (int(pts[0][0]), int(pts[0][1])), (int(pts[1][0]), int(pts[1][1])), (0,255,0), lineThickness)
cv2.line(img1, (int(pts[1][0]), int(pts[1][1])), (int(pts[2][0]), int(pts[2][1])), (0,255,0), lineThickness)
cv2.line(img1, (int(pts[2][0]), int(pts[2][1])), (int(pts[3][0]), int(pts[3][1])), (0,255,0), lineThickness)
cv2.line(img1, (int(pts[3][0]), int(pts[3][1])), (int(pts[0][0]), int(pts[0][1])), (0,255,0), lineThickness)


plt.figure()
plt.imshow(img1)
plt.title("Points Rectangle")
plt.show()

In [9]:

warp = cv2.warpPerspective(img, H, (300,150))
plt.figure()
plt.imshow(warp)
plt.title("After Perspective")
plt.show()


## Removing Affine Distrotion

In [20]:
def remove_affine(img):
    
    plt.figure()
    plt.imshow(img)
    print("Select 4 points in Image")
    pts=plt.ginput(n=4,timeout=0)
    plt.show()
    
    A=np.array([[pts[0][0]*pts[1][0], pts[0][0]*pts[1][1]+pts[0][1]*pts[1][0]],
            [pts[2][0]*pts[3][0], pts[2][0]*pts[3][1]+pts[2][1]*pts[3][0]]])
    b=np.array([-pts[0][1]*pts[1][1],-pts[2][1]*pts[3][1]])
    
    X=np.linalg.inv(A).dot(b)
    
    
    S=np.array([[X[0],X[1]],
            [X[1],1]])
    
    G,V=np.linalg.eig(S)
#     U,G,V=np.linalg.svd(S)
    
#     G_=np.sqrt(np.abs(G))
    
    A=V.dot(np.diag(G)).dot(V.transpose())
#     A=U.dot(np.diag(G_)).dot(V)
    
    Ha=np.array([[A[0,0],A[0,1],0],
             [A[1,0],A[1,1],0],
             [0,0,1]])
    
    Hb=np.linalg.inv(Ha)
    
    return  Hb, pts


In [21]:
Hb,pts=remove_affine(img)


Select 4 points in Image


## Drawing selected points

In [16]:
lineThickness = 2

cv2.line(img1, (int(pts[0][0]), int(pts[0][1])), (int(pts[2][0]), int(pts[2][1])), (0,0,255), lineThickness)
cv2.line(img1, (int(pts[1][0]), int(pts[1][1])), (int(pts[3][0]), int(pts[3][1])), (0,0,255), lineThickness)

plt.figure()
plt.imshow(img1)
plt.figure()

<Figure size 640x480 with 0 Axes>

In [22]:
warpf = cv2.warpAffine(warp, Hb[:2,:], (1000,1000))
plt.figure()
plt.imshow(warpf)
plt.show()
